### Random code/playground while I'm learning the ropes

In [10]:
from cdp_data import CDPInstances, datasets

datasets_df = datasets.get_session_dataset(
	infrastructure_slug=CDPInstances.Seattle,
	start_datetime="2023-08-01",
	store_transcript=True
)

datasets_df

Fetching each model attached to event_ref:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching transcripts:   0%|          | 0/6 [00:00<?, ?it/s]

,session_datetime,session_index,session_content_hash,video_uri,video_start_time,video_end_time,caption_uri,external_source_id,id,key,event,transcript,transcript_path
0,2023-08-10 16:30:00+00:00,0,6e7b96a7887b865491e8706c3fce721b311169533d9416...,https://video.seattle.gov/media/council/fin_08...,None,None,https://www.seattlechannel.org/documents/seatt...,None,e7ef50e46286,session/e7ef50e46286,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
1,2023-08-15 16:30:00+00:00,0,e174b83903970cad3aa7c8b10e79f2cad0daf171535e84...,https://video.seattle.gov/media/council/tran_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,667aaee0ce7f,session/667aaee0ce7f,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
2,2023-08-15 21:00:00+00:00,0,ee91f0df9697a208bba4941b9ed144be407f2da2920200...,https://video.seattle.gov/media/council/counci...,None,None,https://www.seattlechannel.org/documents/seatt...,None,bc815f76977a,session/bc815f76977a,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
3,2023-09-05 16:30:00+00:00,0,5dc9c360ceacf1354ea45a2dcedef2baa61ef37ae402ec...,https://video.seattle.gov/media/council/tran_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,a8f723913165,session/a8f723913165,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
4,2023-09-11 16:30:00+00:00,0,c8c03d4b78b001baaf2ef698a45df2a5202868c359fc43...,https://video.seattle.gov/media/council/econ_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,a14530608f7e,session/a14530608f7e,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
5,2023-09-12 16:30:00+00:00,0,a941c22ec1d46722b5e7dd197df22a07e570b2563365d4...,https://video.seattle.gov/media/council/safe_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,87fdccfcba38,session/87fdccfcba38,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...


In [107]:
from cdp_backend.database import models

event_0 = datasets_df.iloc[0]["event"]
event_0.to_dict()

{'body_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x17d58ba10>,
 'event_datetime': DatetimeWithNanoseconds(2023, 8, 10, 16, 30, tzinfo=datetime.timezone.utc),
 'static_thumbnail_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x17d8be8d0>,
 'hover_thumbnail_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x17d6af990>,
 'agenda_uri': None,
 'minutes_uri': 'https://legistar2.granicus.com/seattle/meetings/2023/8/5578_M_Finance_and_Housing_Committee_23-08-10_Committee_Minutes.pdf',
 'external_source_id': '5578',
 'id': 'f5beff18eb58',
 'key': 'event/f5beff18eb58'}

In [24]:
event_0.body_ref.get()

Okay, I can see I can follow an `Event` to `Body`... but what if I want to get some `Matter`s?

In [47]:
import typing as t
import json
import fireo
from google.cloud.firestore import Client
from google.auth.credentials import AnonymousCredentials

# Fireo has some wacky global nonsense that makes it annoying to use --
# much happier would be to pass an explicit connection instance around, 
# but it mostly works hard to prevent you from thinking about it.
fireo.connection(
	client=Client(
    credentials=AnonymousCredentials(),
    project=CDPInstances.Seattle,
  )	
)

results = list(models.Matter.collection.fetch())
matter_0 = t.cast(models.Matter, results[0])
print(matter_0.to_dict())

{'name': 'CB 120231', 'matter_type': 'Ordinance (Ord)', 'title': 'AN ORDINANCE relating to City employment; authorizing the execution of an agreement between The City of Seattle and Seattle Fire Chiefs Association, International Association of Fire Fighters (IAFF) Local 2898; and ratifying and confirming certain prior acts.', 'external_source_id': '12226', 'id': '0015333ecb6f', 'key': 'matter/0015333ecb6f'}


In [48]:
# Get all the files for a given matter
results = list(models.MatterFile.collection.filter(matter_ref=matter_0.key).fetch())
for matter_file in results:
	assert isinstance(matter_file, models.MatterFile)
	print(matter_file.to_dict())

{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x2a39bf750>, 'name': 'Signed Ordinance 126500', 'uri': 'https://legistar2.granicus.com/seattle/attachments/f2d2066a-9001-435b-bbc4-4c2a1f4edc5b.pdf', 'external_source_id': '32616', 'id': '0e11bcb3a973', 'key': 'matter_file/0e11bcb3a973'}
{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x2a3a28c10>, 'name': 'Summary and Fiscal Note', 'uri': 'https://legistar2.granicus.com/seattle/attachments/68ec5a51-d137-4b75-92b5-7e0a288a6464.docx', 'external_source_id': '31903', 'id': '0f0aaa318f9b', 'key': 'matter_file/0f0aaa318f9b'}
{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x17daf2590>, 'name': 'Att 1 - Memorandum of Agreement', 'uri': 'https://legistar2.granicus.com/seattle/attachments/f22d00d2-71b5-4b5c-a3a4-d504e89e05aa.docx', 'external_source_id': '32288', 'id': 'a007e73739c4', 'key': 'matter_file/a007e73739c4'}
{'matter_ref': <fireo.queries.query_wrapper.ReferenceD

Okay, good. We need to tie all this stuff to events, however. Let's try...

In [50]:
results = list(models.MinutesItem.collection.filter(matter_ref=matter_0.key).fetch())
for minutes_item in results:
  assert isinstance(minutes_item, models.MinutesItem)
  print(minutes_item.to_dict())
minutes_item = t.cast(models.MinutesItem, results[0])

{'name': 'CB 120231', 'description': 'AN ORDINANCE relating to City employment; authorizing the execution of an agreement between The City of Seattle and Seattle Fire Chiefs Association, International Association of Fire Fighters (IAFF) Local 2898; and ratifying and confirming certain prior acts.', 'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x134a75a10>, 'external_source_id': '87641', 'id': '0015333ecb6f', 'key': 'minutes_item/0015333ecb6f'}


In [52]:
results = list(models.EventMinutesItem.collection.filter(minutes_item_ref=minutes_item.key).fetch())
for event_minutes_item in results:
  assert isinstance(event_minutes_item, models.EventMinutesItem)
  print(event_minutes_item.to_dict())
event_minutes_item = t.cast(models.EventMinutesItem, results[0])

{'event_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x134a73e10>, 'minutes_item_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x134a75550>, 'index': 20, 'decision': 'Passed', 'external_source_id': None, 'id': '63d1a3d0ddcc', 'key': 'event_minutes_item/63d1a3d0ddcc'}


In [55]:
event = event_minutes_item.event_ref.get()
event.to_dict()

{'body_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x17daec050>,
 'event_datetime': DatetimeWithNanoseconds(2021, 12, 6, 22, 0, tzinfo=datetime.timezone.utc),
 'static_thumbnail_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x17d523350>,
 'hover_thumbnail_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x17daec0d0>,
 'agenda_uri': None,
 'minutes_uri': 'https://legistar2.granicus.com/seattle/meetings/2021/12/4927_M_City_Council_21-12-06_Full_Council_Meeting_Minutes.pdf',
 'external_source_id': '4927',
 'id': '2adc154d91b0',
 'key': 'event/2adc154d91b0'}

In [108]:
# Prettyprint the event.event_datetime
import datetime
import zoneinfo
seattle_tz = zoneinfo.ZoneInfo("America/Los_Angeles")
print(event_0.event_datetime.astimezone(seattle_tz).strftime("%A, %B %d, %Y %I:%M %p"))

Thursday, August 10, 2023 09:30 AM


Okay, good. We've gone from `Matter` to `Event`. Let's go the other way: given a set of events, find all the related matters.

In [109]:
def matters_for_event(event: models.Event) -> list[models.Matter]:
  # TODO I assume there's a better, more firestore-database-y way to do this?
  event_minutes_items = t.cast(list[models.EventMinutesItem], list(models.EventMinutesItem.collection.filter(event_ref=event.key).order("index").fetch()))
  minutes_items = t.cast(list[models.MinutesItem], [event_minutes_item.minutes_item_ref.get() for event_minutes_item in event_minutes_items])
  matters = t.cast(list[models.Matter], [matter_ref.get() for minutes_item in minutes_items if (matter_ref := minutes_item.matter_ref)])
  return matters

matters = matters_for_event(event_0)
for matter in matters:
  print(matter.to_dict())


{'name': 'Inf 2309', 'matter_type': 'Information Item (Inf)', 'title': 'Seattle Revenue Stabilization Work Group', 'external_source_id': '14450', 'id': '4a5265c81813', 'key': 'matter/4a5265c81813'}


In [110]:
def matter_files_for_matter(matter: models.Matter) -> list[models.MatterFile]:
  results = list(models.MatterFile.collection.filter(matter_ref=matter.key).fetch())
  return [t.cast(models.MatterFile, result) for result in results]

matter_files = matter_files_for_matter(matters[0])
for matter_file in matter_files:
  print(matter_file.to_dict())

{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x134655a10>, 'name': 'Central Staff Memo - Revenue Stabilization Work Group – Process and Next Steps', 'uri': 'https://legistar2.granicus.com/seattle/attachments/6350c245-4167-47b7-a24f-d9b962af85cc.pdf', 'external_source_id': '38380', 'id': '3bd1fff1b31b', 'key': 'matter_file/3bd1fff1b31b'}
{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x2a4c41f10>, 'name': 'Central Staff Memo - General Fund Deficit Historical Analysis', 'uri': 'https://legistar2.granicus.com/seattle/attachments/cb8c865f-95bc-48ff-877c-2f477493f14c.pdf', 'external_source_id': '38379', 'id': '9c60d136c0f6', 'key': 'matter_file/9c60d136c0f6'}
{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x2a373bb50>, 'name': 'Transit Riders Union Revenue Options Report', 'uri': 'https://legistar2.granicus.com/seattle/attachments/34ef34c2-9327-4060-b76e-128f4c023737.pdf', 'external_source_id': '38288', 'id': 'e

This looks reasonable, if incomplete. In particular, visiting this item on Legistar today (September 27, 2023), I see:

1. Work Group Report
2. Central Staff Memo - General Fund Deficit Historical Analysis
3. Central Staff Memo - Revenue Stablization Work Group - Process And Next Steps
4. Transit Riders Union Revenue Options Report

Looks like we're missing the `Work Group Report`. Was it posted later to legistar, after the crawler ran? Does the crawler know to revisit older items?

See https://seattle.legistar.com/LegislationDetail.aspx?ID=6309644&GUID=0C784283-157E-4CA2-945F-86BEF744C6A1&Options=&Search=

Let's try for a different event:

In [113]:
event_1 = datasets_df.iloc[1]["event"]
print(event_1.event_datetime.astimezone(seattle_tz).strftime("%A, %B %d, %Y %I:%M %p"))
matters = matters_for_event(event_1)
for matter in matters:
  print(matter.to_dict())
matter_0 = matters[0]
print("---")
print(matter_0.to_dict())
matter_files = matter_files_for_matter(matter_0)
for matter_file in matter_files:
  print(matter_file.to_dict())

Tuesday, August 15, 2023 09:30 AM
{'name': 'CB 120639', 'matter_type': 'Council Bill (CB)', 'title': 'AN ORDINANCE relating to acceptance of funding from non-City sources; authorizing the General Manager and CEO of Seattle Public Utilities to accept specified loans to partially finance costs related to the construction of the South Thornton Natural Drainage System Project; and ratifying and confirming certain prior acts.', 'external_source_id': '14345', 'id': '066f16cdbe62', 'key': 'matter/066f16cdbe62'}
{'name': 'CF 314496', 'matter_type': 'Clerk File (CF)', 'title': 'Petition of the Central Puget Sound Regional Transit Authority, for the vacation of a portion of the alley in block bounded by NE 45th Street, Roosevelt Way NE, NE 47th Street, and 11th Avenue NE, at 1000 NE 45th Street.', 'external_source_id': '13129', 'id': 'bcfae19095b7', 'key': 'matter/bcfae19095b7'}
{'name': 'CB 120625', 'matter_type': 'Council Bill (CB)', 'title': 'AN ORDINANCE establishing additional uses for auto

Okay, so this is good, with one key exception: **we don't have the text of the matter_0 bill itself**.

Looks like the CDP Seattle scraper needs to be improved here. Unfortunately, the full text of bills and ordinances is not always added as an *attachment*. Rather, it's often only available on the bill's page itself. See here:

https://seattle.legistar.com/LegislationDetail.aspx?ID=6309305&GUID=AB0410F6-5EBD-4360-925B-0E875CA4F525&Options=&Search=&FullText=1

Because the underlying CDP data model does not appear to keep scraped URLs in all cases, it may prove difficult to get the full text of the bill. I see the `external_source_id` in `Matter`, but I'll need to see what the Seattle scraper is actually doing.

More than that: with the current data model, there's no place to *store* the full text of the bill once we have it. The `MatterFile` model simply links to the original document on (say) Legistar. That's fine, but what exactly should the link be in the example above? That link is a full HTML page, only some of which is the text of the council bill. (Maybe that's good enough?)

Of course, this is just Seattle! Maybe we *do* get council bill text for other councils. I suppose we should try. :-)

In [120]:
charlotte_datasets_df = datasets.get_session_dataset(
	infrastructure_slug=CDPInstances.Charlotte,
	start_datetime="2023-08-01",
	store_transcript=True
)

charlotte_datasets_df

Fetching each model attached to event_ref:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching transcripts:   0%|          | 0/3 [00:00<?, ?it/s]

,session_datetime,session_index,session_content_hash,video_uri,video_start_time,video_end_time,caption_uri,external_source_id,id,key,event,transcript,transcript_path
0,2023-08-07 21:00:00+00:00,0,c1103c9cfd4ac74b44ac0c2b1fb67035193c028601cd38...,https://archive-video.granicus.com/charlottenc...,None,None,None,None,1bf4677814a3,session/1bf4677814a3,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
1,2023-08-21 21:00:00+00:00,0,29f37dc5525df61351168e6bcbc02471c75c55ee2e74c0...,https://archive-video.granicus.com/charlottenc...,None,None,None,None,b0bc185b2f9f,session/b0bc185b2f9f,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...
2,2023-09-05 22:00:00+00:00,0,50ae50f023e5d4e68904b871fe479e2479f6a1744fc381...,https://archive-video.granicus.com/charlottenc...,None,None,None,None,563528c86f04,session/563528c86f04,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/dave/dev/cdp/summarize/cdp-datasets/cdp...


In [135]:
charlotte_event_0 = charlotte_datasets_df.iloc[1]["event"]
charlotte_event_0.to_dict()

{'body_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x2a374b950>,
 'event_datetime': DatetimeWithNanoseconds(2023, 8, 21, 21, 0, tzinfo=datetime.timezone.utc),
 'static_thumbnail_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x2a374b650>,
 'hover_thumbnail_ref': <fireo.queries.query_wrapper.ReferenceDocLoader at 0x2a3749910>,
 'agenda_uri': None,
 'minutes_uri': None,
 'external_source_id': '1600',
 'id': '62d3209d1f1c',
 'key': 'event/62d3209d1f1c'}

In [136]:
charlotte_tz = zoneinfo.ZoneInfo("America/New_York")
print(charlotte_event_0.event_datetime.astimezone(charlotte_tz).strftime("%A, %B %d, %Y %I:%M %p"))

Monday, August 21, 2023 05:00 PM


In [137]:
matters = matters_for_event(charlotte_event_0)
for matter in matters:
  print(matter.to_dict())

{'name': '15-21016', 'matter_type': 'Dinner Briefing', 'title': '5:00 P.M. CITY COUNCIL ZONING MEETING\nCHARLOTTE-MECKLENBURG GOVERNMENT CENTER,\nREGULAR MEETING HOSTED FROM COUNCIL CHAMBERS', 'external_source_id': '24304', 'id': 'f0a108fcd648', 'key': 'matter/f0a108fcd648'}
{'name': '15-21018', 'matter_type': 'Zoning Decision', 'title': 'Rezoning Petition: 2021-198 by Nest Home Communities, LLC', 'external_source_id': '24306', 'id': '217d989ce25e', 'key': 'matter/217d989ce25e'}
{'name': '15-21026', 'matter_type': 'Zoning Hearing', 'title': 'Rezoning Petition: 2022-151 by Rayna Properties, LLC', 'external_source_id': '24314', 'id': '6d14796195d0', 'key': 'matter/6d14796195d0'}
{'name': '15-21030', 'matter_type': 'Zoning Decision', 'title': 'Rezoning Petition: 2022-048 by Tribute Companies, Inc.', 'external_source_id': '24318', 'id': 'fa8b5fecb4cd', 'key': 'matter/fa8b5fecb4cd'}
{'name': '15-21022', 'matter_type': 'Zoning Decision', 'title': 'Rezoning Petition: 2022-099 by Levine Proper

In [124]:
matter_files = matter_files_for_matter(matters[0])
for matter_file in matter_files:
  print(matter_file.to_dict())

{'matter_ref': <fireo.queries.query_wrapper.ReferenceDocLoader object at 0x2a3924b90>, 'name': 'Charlotte Commmunity Capital Fund Presentation', 'uri': 'https://charlottenc.legistar1.com/charlottenc/attachments/5d4b4550-c229-4e82-a9a3-a2a20ef6d8bc.pdf', 'external_source_id': '36737', 'id': '667deaaae08f', 'key': 'matter_file/667deaaae08f'}
